<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Trades_in_SugarScape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Mesa (Need to do with Quartsite release)

In [ ]:
!pip install -e git+https://github.com/projectmesa/mesa#egg=mesa

Obtaining mesa from git+https://github.com/projectmesa/mesa#egg=mesa
  Cloning https://github.com/projectmesa/mesa to ./src/mesa
  Running command git clone -q https://github.com/projectmesa/mesa /content/src/mesa
     |████████████████████████████████| 64 kB 3.2 MB/s 
  Running setup.py develop for mesa


## Import the Depedencies 
- Restart Runtime if working from dependencies #Should be removed once Mesa 0.10.0 released
- uplodad file (show uploading traderscape creation)

In [ ]:
import json 

# Opening JSON file
f = open('SugarSpiceScape.json')
 
# returns JSON object as
# a dictionary
land = json.load(f)
land

{'(13, 13)': [4, 0],
 '(14, 12)': [4, 0],
 '(14, 13)': [4, 0],
 '(14, 14)': [4, 0],
 '(15, 13)': [4, 0],
 '(12, 13)': [4, 0],
 '(13, 12)': [4, 0],
 '(13, 14)': [4, 0],
 '(14, 11)': [4, 0],
 '(14, 15)': [4, 0],
 '(15, 12)': [4, 0],
 '(15, 14)': [4, 0],
 '(16, 13)': [4, 0],
 '(11, 13)': [4, 0],
 '(12, 11)': [4, 0],
 '(12, 12)': [4, 0],
 '(12, 14)': [4, 0],
 '(12, 15)': [4, 0],
 '(13, 11)': [4, 0],
 '(13, 15)': [4, 0],
 '(14, 10)': [4, 0],
 '(14, 16)': [4, 0],
 '(15, 11)': [4, 0],
 '(15, 15)': [4, 0],
 '(16, 11)': [4, 0],
 '(16, 12)': [4, 0],
 '(16, 14)': [4, 0],
 '(16, 15)': [4, 0],
 '(17, 13)': [4, 0],
 '(10, 13)': [3, 1],
 '(11, 11)': [3, 1],
 '(11, 12)': [3, 1],
 '(11, 14)': [3, 1],
 '(11, 15)': [3, 1],
 '(12, 10)': [3, 1],
 '(12, 16)': [3, 1],
 '(13, 10)': [3, 1],
 '(13, 16)': [3, 1],
 '(14, 9)': [3, 1],
 '(14, 17)': [3, 1],
 '(15, 10)': [3, 1],
 '(15, 16)': [3, 1],
 '(16, 10)': [3, 1],
 '(16, 16)': [3, 1],
 '(17, 11)': [3, 1],
 '(17, 12)': [3, 1],
 '(17, 14)': [3, 1],
 '(17, 15)': [

## Show pics of what was created 

- show colored
- show numbers not zoomed
- show zoomed

## Build SugarScape 1 agentize the Landscape 

  - convert numbers into sugar and spice agents
  - build the agents

In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid

In [ ]:
class ResourceAgent(Agent):
    
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        self.pos = unique_id
        self.value_sug = attributes[0]
        self.value_spice = attributes[1]
        self.max_sug = attributes[0]
        self.max_spice = attributes[1]
        self.regrow = 1
        self.type = "resource"
        
    def __str__(self):
        return "Resource" #pro-tip

In [ ]:
class TraderScape(Model):

  def __init__(self,landscape, height, width): 
    
    self.height = height
    self.width = width
    self.grid = MultiGrid(self.height, self.width, torus=True)  #New  
    
    #NEW
    #places resources from landscape on grid
    for location, resources in land.items(): 
        #convert coordinates from string
        loc = eval(location)
        resource =  ResourceAgent(loc, self, resources)
        #print(resource) # do first show how the string works
        print(resource.value_sug) #do second
        self.grid.place_agent(resource, (resource.pos[0], resource.pos[1]))

In [ ]:
#Test
TraderScape(land, 50,50)

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
3
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
1
3
1
3
1
3
1
3
1
2
1
3
1
3
1
3
1
3
1
1
3
3
1
1


## Build SugarScape 3 Create The traders